# Рекомендательные системы

## _Описание задачи_

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо **показывать** не последние просмотренные пользователем товары, а те товары **из просмотренных**, которые он наиболее вероятно **купит**. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе **А/В теста**, т.к. по доходу от продаж **статзначимость будет достигаться _дольше_** из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по **частоте просмотров** и по **частоте покупок**. Эти бейзлайны, с одной стороны, могут помочь вам **грубо оценить** возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

## _Входные данные_

Вам дается **две** выборки с пользовательскими сессиями - **id-шниками просмотренных** и **id-шниками купленных** товаров. Одна выборка будет использоваться **для обучения** (оценки популярностей товаров), а другая - **для теста**.

В файлах записаны сессии **по одной в каждой строке**. Формат сессии: **id просмотренных** товаров через "**,**" затем идёт "**;**" после чего следуют **id купленных товаров** (если такие имеются), разделённые запятой. Например, **1,2,3,4;** или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все **различные**.

## _Важно:_

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k** / **precision@k**.

### Загрузим необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from collections import defaultdict

### загрузим данные

In [2]:
train = pd.read_csv('data/coursera_sessions_train.txt', sep=';', header=None, names=['viewed', 'bought'])
test = pd.read_csv('data/coursera_sessions_test.txt', sep=';', header=None, names=['viewed', 'bought'])

In [3]:
train.head(10)

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


In [4]:
test.head(10)

,viewed,bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN
5,"43,44,43,45,43,45,43,46",NaN
6,"50,51,47,52,49,53,54,55,56,57,58",NaN
7,"63,68,69,70,66,61,59,61,66,68","66,63"
8,75,NaN
9,"79,80,81,82,83",NaN


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
viewed    50000 non-null object
bought    3608 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
viewed    50000 non-null object
bought    3665 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


#### предобработка входных данных

In [7]:
%%time
train = train.applymap(lambda s: s.strip().split(',') if s is not np.nan else np.nan)
test = test.applymap(lambda s: s.strip().split(',') if s is not np.nan else np.nan)

Wall time: 614 ms


In [8]:
train.head(10)

,viewed,bought
0,"[0, 1, 2, 3, 4, 5]",NaN
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
2,"[16, 17, 18, 19, 20, 21]",NaN
3,"[24, 25, 26, 27, 24]",NaN
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN
5,[42],NaN
6,"[47, 48, 49]",NaN
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
8,"[71, 72, 73, 74]",NaN
9,"[76, 77, 78]",NaN


In [9]:
test.head()

,viewed,bought
0,"[6, 7, 8]",NaN
1,"[13, 14, 15]",NaN
2,"[22, 23]",NaN
3,"[28, 29, 30, 31, 32, 33]",NaN
4,"[40, 41]",NaN


## _Задание_
### 1.
На обучении постройте частоты появления **id** в просмотренных и в купленных (**id** может несколько раз появляться в просмотренных, все появления надо учитывать)

Сформируем соответствующую функцию, которая по выборке строит словари частот появления **id** в просмотренных и купленных:

In [10]:
def freq_dict(data):   
   
    freq = {} # инициализация словаря словарей
    freq['viewed'] = defaultdict(int) # инициализация словаря частот по просмотрам
    freq['bought'] = defaultdict(int) # инициализация словаря частот по покупкам
    
    for i, row in tqdm_notebook(data.iterrows(), total=len(data)):
        for col in ['viewed', 'bought']:
            if row[col] is not np.nan:
                for ID in row[col]:
                    freq[col][ID] += 1
        
    return freq

пройдемся по **train** и сформируем **словарь частот** появления **id** в **viewed** и **bought**

In [11]:
freq = freq_dict(train)

A Jupyter Widget

In [12]:
len(freq['viewed']), len(freq['bought'])

(77064, 4479)

### 2.
Реализуйте два алгоритма рекомендаций:    

- сортировка просмотренных **id** по **популярности** (частота появления в **просмотренных**),
- сортировка просмотренных **id** по **покупаемости** (частота появления в **покупках**).

Добавим в данные (train и test) колонки с рекомендациями (префикс "**recomm_**")

Для этого сформируем соответствующую функцию **recommended**  для рекомендаций в текущей сессии.

In [13]:
def recommended(sess, freq, kind, k):
    '''
    sess - список id для сессии, каждый id - строковый
    freq - словарь словарей частот
    kind - ключ, с которым в freq создается соответствующий словарь, принимает значения ['viewed', 'bought']
    k - число рекомендаций
    '''
    # сопоставляем список рекомендаций с параметрами, по которым потом сортируем, а именно: частота и момент появления
    recomm = [[ID, ((freq[kind][ID] if ID in freq[kind] else 0), sess.index(ID))] for ID in set(sess)]
    
    # сортируем (по 2м(!) параметрам, причем разнонаправлено, что обеспечивает "-")
    recomm_sorted = list(map(lambda x: x[0], sorted(recomm, key=lambda x: [-x[1][0], x[1][1]]))) 
    
    # выводим не более k рекомендаций
    return recomm_sorted[:k]

Теперь добавим в данные (train и test) колонки с **k** рекомендациями (префикс "**recomm_**") по 2-м вариантам словарей частот

In [14]:
%%time
for df in [train, test]:
    for k in[1, 5]:
        for kind in ['viewed', 'bought']:
            df['recomm_' + kind + '_k=' + str(k)] = df['viewed'].apply(lambda sess: recommended(sess, freq, kind, k))        

Wall time: 6.42 s


In [15]:
train.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5
0,"[0, 1, 2, 3, 4, 5]",NaN,[4],[5],"[4, 2, 3, 0, 1]","[5, 0, 1, 2, 3]"
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN,[12],[9],"[12, 9, 10, 11]","[9, 10, 11, 12]"
2,"[16, 17, 18, 19, 20, 21]",NaN,[17],[17],"[17, 20, 16, 19, 18]","[17, 16, 18, 19, 20]"
3,"[24, 25, 26, 27, 24]",NaN,[27],[24],"[27, 24, 26, 25]","[24, 25, 26, 27]"
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN,[35],[38],"[35, 34, 36, 37, 38]","[38, 34, 35, 36, 37]"


In [16]:
test.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5
0,"[6, 7, 8]",NaN,[7],[8],"[7, 6, 8]","[8, 6, 7]"
1,"[13, 14, 15]",NaN,[13],[13],"[13, 14, 15]","[13, 14, 15]"
2,"[22, 23]",NaN,[22],[22],"[22, 23]","[22, 23]"
3,"[28, 29, 30, 31, 32, 33]",NaN,[28],[28],"[28, 33, 30, 29, 32]","[28, 29, 30, 32, 31]"
4,"[40, 41]",NaN,[40],[40],"[40, 41]","[40, 41]"


### Более компактный и более быстрый способ создания колонок рекомендаций:

In [17]:
%%time
for df in [train, test]:
    for k in[1, 5]:
        for kind in ['viewed', 'bought']:
            df['recomm_' + kind + '_k=' + str(k)] = df['viewed']. \
                                    apply(lambda sess: sorted(set(sess), key=lambda ID: (-freq[kind][ID], sess.index(ID)))[:k])

# через set() убираем повторяющиеся id и затем сортируем по 2-м ключам. После чего оставляем min(k, len(set(sess)) номеров id

Wall time: 4.64 s


In [18]:
train.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5
0,"[0, 1, 2, 3, 4, 5]",NaN,[4],[5],"[4, 2, 3, 0, 1]","[5, 0, 1, 2, 3]"
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN,[12],[9],"[12, 9, 10, 11]","[9, 10, 11, 12]"
2,"[16, 17, 18, 19, 20, 21]",NaN,[17],[17],"[17, 20, 16, 19, 18]","[17, 16, 18, 19, 20]"
3,"[24, 25, 26, 27, 24]",NaN,[27],[24],"[27, 24, 26, 25]","[24, 25, 26, 27]"
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN,[35],[38],"[35, 34, 36, 37, 38]","[38, 34, 35, 36, 37]"


### 3.
Для данных алгоритмов выпишите через пробел **AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### _учтем:_

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k** / **precision@k**.

Сессии, в которых пользователь **ничего не купил**, из оценки качества **исключаем**:

In [19]:
train_buy = train.dropna().copy()
test_buy = test.dropna().copy()

In [20]:
train_buy.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]",[63],[60],"[63, 64, 60, 61, 65]","[60, 63, 67, 59, 61]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],[85],[86],"[85, 93, 89, 90, 84]","[86, 85, 93, 84, 87]"
19,"[138, 198, 199, 127]",[199],[127],[138],"[127, 138, 198, 199]","[138, 199, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],[303],[303],"[303, 306, 304, 307, 309]","[303, 304, 305, 306, 307]"
33,"[352, 353, 352]",[352],[352],[352],"[352, 353]","[352, 353]"


Считаем соответствующие метрики по сессиям (рядам):

In [21]:
%%time
for df in [train_buy, test_buy]:
    for i, row in tqdm_notebook(df.iterrows(), total=len(df)):
        for kind in ['viewed', 'bought']:
            for k in[1, 5]:
                for score in ['Recall', 'Precision']:
                    col = score + '_' + kind + '_at_' + str(k)
                    col_rec = 'recomm_' + kind + '_k=' + str(k)
                    
                    df.loc[i, col] = float(len(set(row['bought']) & set(row[col_rec]))) / \
                                     (k if  score == 'Precision' else len(row['bought']))

A Jupyter Widget

A Jupyter Widget


Wall time: 39.7 s


In [22]:
train_buy.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5,Recall_viewed_at_1,Precision_viewed_at_1,Recall_viewed_at_5,Precision_viewed_at_5,Recall_bought_at_1,Precision_bought_at_1,Recall_bought_at_5,Precision_bought_at_5
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]",[63],[60],"[63, 64, 60, 61, 65]","[60, 63, 67, 59, 61]",0.333333,1.0,0.666667,0.4,0.333333,1.0,1.0,0.6
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],[85],[86],"[85, 93, 89, 90, 84]","[86, 85, 93, 84, 87]",0.000000,0.0,0.000000,0.0,1.000000,1.0,1.0,0.2
19,"[138, 198, 199, 127]",[199],[127],[138],"[127, 138, 198, 199]","[138, 199, 127, 198]",0.000000,0.0,1.000000,0.2,0.000000,0.0,1.0,0.2
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],[303],[303],"[303, 306, 304, 307, 309]","[303, 304, 305, 306, 307]",1.000000,1.0,1.000000,0.2,1.000000,1.0,1.0,0.2
33,"[352, 353, 352]",[352],[352],[352],"[352, 353]","[352, 353]",1.000000,1.0,1.000000,0.2,1.000000,1.0,1.0,0.2


In [23]:
test_buy.head()

,viewed,bought,recomm_viewed_k=1,recomm_bought_k=1,recomm_viewed_k=5,recomm_bought_k=5,Recall_viewed_at_1,Precision_viewed_at_1,Recall_viewed_at_5,Precision_viewed_at_5,Recall_bought_at_1,Precision_bought_at_1,Recall_bought_at_5,Precision_bought_at_5
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]",[63],[63],"[63, 68, 66, 61, 59]","[63, 68, 69, 70, 66]",0.5,1.0,1.0,0.4,0.5,1.0,1.0,0.4
14,"[158, 159, 160, 159, 161, 162]",[162],[158],[158],"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]",0.0,0.0,1.0,0.2,0.0,0.0,1.0,0.2
19,"[200, 201, 202, 203, 204]","[201, 205]",[204],[204],"[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]",0.0,0.0,0.5,0.2,0.0,0.0,0.5,0.2
34,"[371, 372, 371]","[371, 373]",[371],[371],"[371, 372]","[371, 372]",0.5,1.0,0.5,0.2,0.5,1.0,0.5,0.2
40,[422],[422],[422],[422],[422],[422],1.0,1.0,1.0,0.2,1.0,1.0,1.0,0.2


Сохраним на всякий случай полученные данные

In [24]:
train.to_csv('data/train_processed.csv')
test.to_csv('data/test_processed.csv')
train_buy.to_csv('data/train_buy.csv')
test_buy.to_csv('data/test_buy.csv')

Посчитаем средние метрики через усреднения соответствующих локальных метрик по всем сессиям

In [25]:
train_metrica = train_buy[['Recall_viewed_at_1', 'Precision_viewed_at_1', 'Recall_viewed_at_5', 'Precision_viewed_at_5', 
                           'Recall_bought_at_1', 'Precision_bought_at_1', 'Recall_bought_at_5', 'Precision_bought_at_5']] \
                           .mean(axis=0).apply(lambda x: round(x, 2))
train_metrica

Recall_viewed_at_1       0.44
Precision_viewed_at_1    0.51
Recall_viewed_at_5       0.82
Precision_viewed_at_5    0.21
Recall_bought_at_1       0.69
Precision_bought_at_1    0.80
Recall_bought_at_5       0.93
Precision_bought_at_5    0.25
dtype: float64

In [26]:
test_metrica = test_buy[['Recall_viewed_at_1', 'Precision_viewed_at_1', 'Recall_viewed_at_5', 'Precision_viewed_at_5', 
                         'Recall_bought_at_1', 'Precision_bought_at_1', 'Recall_bought_at_5', 'Precision_bought_at_5']] \
                         .mean(axis=0).apply(lambda x: round(x, 2))
test_metrica

Recall_viewed_at_1       0.42
Precision_viewed_at_1    0.48
Recall_viewed_at_5       0.80
Precision_viewed_at_5    0.20
Recall_bought_at_1       0.46
Precision_bought_at_1    0.53
Recall_bought_at_5       0.82
Precision_bought_at_5    0.21
dtype: float64

Запишем ответы

In [27]:
with open('answers/viewed_popularity_train.txt', 'w') as file:
    file.write(' '.join(np.array(train_metrica).astype(str)[:4]))

with open('answers/viewed_popularity_test.txt', 'w') as file:
    file.write(' '.join(np.array(test_metrica).astype(str)[:4]))

with open('answers/bought_popularity_train.txt', 'w') as file:
    file.write(' '.join(np.array(train_metrica).astype(str)[4:]))

with open('answers/bought_popularity_test.txt', 'w') as file:
    file.write(' '.join(np.array(test_metrica).astype(str)[4:]))

_**Дополнительные вопросы**_

1. Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров **recall@k** меняется. Подумайте, как оценить минимальное и максимальное значение **recall@k** в зависимости от правила сортировки.


2. Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.